Solve Synthetic Tides
=====================

This ({nb-download}`notebook <Solve-Synthetic-Tides.ipynb>`) solving for the harmonic constants for a tidal time series at a given location

```{important}
*Need to download tide model prior to running this notebook.*  
```

OTIS format tidal solutions provided by Oregon State University and ESR  
- [http://volkov.oce.orst.edu/tides/region.html](http://volkov.oce.orst.edu/tides/region.html) 
- [https://www.esr.org/research/polar-tide-models/list-of-polar-tide-models/](https://www.esr.org/research/polar-tide-models/list-of-polar-tide-models/)
- [ftp://ftp.esr.org/pub/datasets/tmd/](ftp://ftp.esr.org/pub/datasets/tmd/)  

Global Tide Model (GOT) solutions provided by Richard Ray at GSFC  
- [https://earth.gsfc.nasa.gov/geo/data/ocean-tide-models](https://earth.gsfc.nasa.gov/geo/data/ocean-tide-models)

Finite Element Solution (FES) provided by AVISO  
- [https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes.html](https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes.html)

## Python Dependencies
 - [h5netcdf: Pythonic interface to netCDF4 via h5py](https://h5netcdf.org/)
 - [ipyleaflet: Jupyter / Leaflet bridge enabling interactive maps](https://github.com/jupyter-widgets/ipyleaflet)  
 - [matplotlib: Python 2D plotting library](https://matplotlib.org/)  
 - [numpy: Scientific Computing Tools For Python](https://www.numpy.org)  
 - [pyproj: Python interface to PROJ library](https://pypi.org/project/pyproj/)  
 - [scipy: Scientific Tools for Python](https://www.scipy.org/)  
 - [xarray: N-D labeled arrays and datasets in Python](https://docs.xarray.dev/en/stable/) 

## Program Dependencies

- `astro.py`: computes the basic astronomical mean longitudes  
- `constituents.py`: calculates constituent parameters and nodal arguments  
- `io.model.py`: retrieves tide model parameters for named tide models
- `io.OTIS.py`: extract tidal harmonic constants from OTIS tide models  
- `io.ATLAS.py`: extract tidal harmonic constants from ATLAS netCDF4 tide models  
- `io.GOT.py`: extract tidal harmonic constants from GOT tide models  
- `io.FES.py`: extract tidal harmonic constants from FES tide models  
- `predict.py`: predict tidal values using harmonic constants  
- `solve.py`: estimates the harmonic constants for ocean tides
- `time.py`: utilities for calculating time operations
- `utilities.py`: download and management utilities for files

```{note}
This notebook uses Jupyter widgets to set parameters for calculating the tidal maps.  
```

## Load modules

In [ ]:
from __future__ import print_function

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import IPython.display
import ipywidgets

# import tide programs
import pyTMD.tools
import timescale.time

# autoreload
%load_ext autoreload
%autoreload 2

## Set parameters for program

- Model directory  
- Tide model for synthetic  

In [ ]:
# available model list
model_list = sorted(pyTMD.io.model.ocean_elevation())
# display widgets for setting directory and model
TMDwidgets = pyTMD.tools.widgets()
TMDwidgets.model.options = model_list
TMDwidgets.model.value = 'GOT4.10_nc'
TMDwidgets.VBox([
    TMDwidgets.directory,
    TMDwidgets.model,
])

## Select location for tide synthetic 

In [ ]:
# default coordinates to use
LAT,LON = (-76.0, -40.0)
m = pyTMD.tools.leaflet(center=(LAT,LON), zoom=3,
    zoom_control=True, marker_control=True,
    attribution=False)
# show map
m.map

## HTML table with outputs

In [ ]:
table = ipywidgets.HTML()
display(table)

## Calculate and plot solution

In [ ]:
# get model parameters and open dataset
model = pyTMD.io.model(TMDwidgets.directory.value).from_database(TMDwidgets.model.value)
ds = model.open_dataset(group='z')
# list of model constituents
c = ds.tmd.constituents

# convert time to days relative to Jan 1, 1992 (48622 MJD)
minutes = np.arange(366*1440)
ts = timescale.from_calendar(2000, 1, 1, minute=minutes)

# delta time (TT - UT1)
if model.format in ('GOT-ascii', 'GOT-netcdf', 'FES-netcdf'):
    DELTAT = ts.tt_ut1
elif (model.format == 'FES-netcdf'):
    DELTAT = np.zeros_like(ts.tide)

# update the tide solution
def update_tide_solution(*args):
    # leaflet location
    LAT,LON = np.copy(m.marker.location)
    # verify longitudes
    LON = m.wrap_longitudes(LON)
    # convert to model grid coordinates
    x, y = ds.tmd.transform(LON, LAT)
    # interpolate to grid points and convert to centimeters
    local = ds.tmd.interp(x, y, extrapolate=True)
    local = local.tmd.to_units('cm')
    # predict tidal elevations at time
    TIDE = local.tmd.predict(ts.tide, deltat=DELTAT,
        corrections=model.corrections)
    # infer minor constituents and add to major components
    TIDE += local.tmd.infer(ts.tide, deltat=DELTAT,
        corrections=model.corrections)

    # solve for harmonic constants
    dsolve = pyTMD.solve.constants(ts.tide, TIDE.values, c,
        deltat=DELTAT, corrections=model.corrections,
        infer_minor=True)

    # create a HTML table with values
    # HTML table header
    table.value = """<table>
		<thead>
		<tr>
			<th style="text-align:center; padding: 10px">Constituent</th>
			<th style="text-align:center; padding: 10px">Original Amplitude</th>
			<th style="text-align:center; padding: 10px">Original Phase</th>
			<th style="text-align:center; padding: 10px">Solution Amplitude</th>
			<th style="text-align:center; padding: 10px">Solution Phase</th>
		</tr>
		</thead>
		<tbody>
    """
    # print the original and solution amplitudes and phases to table
    for i,con in enumerate(c):
        table.value += f"""
            <tr>
			    <td style="text-align:center">{con}</td>
			    <td style="text-align:center">{local[con].tmd.amplitude:0.1f}cm</td>
                <td style="text-align:center">{local[con].tmd.phase:0.1f}\u00b0</td>
			    <td style="text-align:center">{dsolve[con].tmd.amplitude:0.1f}cm</td>
                <td style="text-align:center">{dsolve[con].tmd.phase:0.1f}\u00b0</td>
            </tr>
        """
    # close HTML table
    table.value += """
        </tbody>
    </table>
    """

# run tide prediction and solution at initial location
update_tide_solution()
# watch marker location for changes
m.marker_text.observe(update_tide_solution)